In [ ]:
# C01-aws-client

In [ ]:
# Install AWS Client and boto3 

In [ ]:
! pip install awscli

In [ ]:
! pip install boto3 

In [ ]:
### Example of using Boto3 to create an S3 bucket - make a small file and upload it ##

In [ ]:
# Import Libraries #
import boto3
from botocore.exceptions import ClientError
import time

In [ ]:
# Step 1: Create a sample text file
filename = "sample_file.txt"
with open(filename, "w") as f:
    f.write("Hello from boto3!\n")
    f.write("This is a sample file uploaded to S3.\n")
    f.write("Created on: " + time.strftime("%Y-%m-%d %H:%M:%S"))

print(f"✓ Created {filename}")

In [ ]:
# Step 2: Create an S3 bucket
# Note: Bucket names must be globally unique across ALL AWS accounts
bucket_name = f"my-test-bucket-{int(time.time())}"  # Adding timestamp for uniqueness
region = "us-west-2"  # Change to your preferred region

s3_client = boto3.client('s3', region_name=region)

try:
    # Create bucket
    if region == "us-east-1":
        # us-east-1 doesn't need LocationConstraint
        s3_client.create_bucket(Bucket=bucket_name)
    else:
        s3_client.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={'LocationConstraint': region}
        )
    print(f"✓ Created bucket: {bucket_name}")
    
except ClientError as e:
    print(f"Error creating bucket: {e}")
    exit(1)

In [ ]:
# Step 3: Upload the file to S3
try:
    s3_client.upload_file(filename, bucket_name, filename)
    print(f"✓ Uploaded {filename} to s3://{bucket_name}/{filename}")
    
    # Verify upload by listing objects
    response = s3_client.list_objects_v2(Bucket=bucket_name)
    print(f"\nFiles in bucket:")
    for obj in response.get('Contents', []):
        print(f"  - {obj['Key']} ({obj['Size']} bytes)")
        
except ClientError as e:
    print(f"Error uploading file: {e}")

In [ ]:
## Review the resources created ##
# aws s3 ls 
# aws s3 ls s3://your-bucket

In [ ]:
# Step 4: Clean up - Delete the file and bucket
print("\n--- Cleaning up ---")
try:
    # First, delete all objects in the bucket (required before deleting bucket)
    response = s3_client.list_objects_v2(Bucket=bucket_name)
    if 'Contents' in response:
        for obj in response['Contents']:
            s3_client.delete_object(Bucket=bucket_name, Key=obj['Key'])
            print(f"✓ Deleted {obj['Key']} from bucket")
    
    # Now delete the bucket
    s3_client.delete_bucket(Bucket=bucket_name)
    print(f"✓ Deleted bucket: {bucket_name}")
    
except ClientError as e:
    print(f"Error during cleanup: {e}")

In [ ]:
# Check that your resources have been removed #

In [ ]:
## End of Noteboook ##